In [8]:
# LISBON GEOSPATIAL ANALYSIS
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

# Set Plotly default theme
import plotly.io as pio
pio.templates.default = "plotly_white"

def normalize_freguesia(df, col_name):
    if col_name in df.columns:
        df['Freguesia_Norm'] = df[col_name].str.strip().str.upper()
    return df

def plot_choropleth(gdf, column, title, cmap='viridis', vmin=None, vmax=None, categorical=False):
    """
    Consistent plotting function using Plotly for interactivity.
    """
    # Check if column exists
    if column not in gdf.columns:
        print(f"Column {column} not found. Skipping.")
        return

    # Reproject to WGS84 for Mapbox
    gdf_4326 = gdf.to_crs("EPSG:4326")
    
    # Create Hover Data
    hover_data = {'Freguesia_Norm': True, column: True}
    
    # Handle Colormap mapping (Matplotlib names to Plotly names/lists)
    color_scale = cmap
    if cmap == 'RdBu': color_scale = 'RdBu'
    elif cmap == 'OrRd': color_scale = 'OrRd'
    elif cmap == 'YlGn': color_scale = 'YlGn'
    elif cmap == 'Purples': color_scale = 'Purples'
    elif cmap == 'Blues': color_scale = 'Blues'
    elif cmap == 'Greens': color_scale = 'Greens'
    elif cmap == 'magma': color_scale = 'Magma'
    elif cmap == 'viridis': color_scale = 'Viridis'
    
    # Determine range
    range_color = [vmin, vmax] if vmin is not None and vmax is not None else None

    if categorical:
        fig = px.choropleth_mapbox(
            gdf_4326,
            geojson=gdf_4326.geometry,
            locations=gdf_4326.index,
            color=column,
            hover_name='Freguesia_Norm',
            hover_data={column: True},
            title=title,
            mapbox_style="carto-positron",
            center={"lat": 38.7223, "lon": -9.1393},
            zoom=11,
            opacity=0.7,
            color_discrete_sequence=px.colors.qualitative.Bold
        )
    else:
        fig = px.choropleth_mapbox(
            gdf_4326,
            geojson=gdf_4326.geometry,
            locations=gdf_4326.index,
            color=column,
            hover_name='Freguesia_Norm',
            hover_data={column: True},
            title=title,
            mapbox_style="carto-positron",
            center={"lat": 38.7223, "lon": -9.1393},
            zoom=11,
            opacity=0.7,
            color_continuous_scale=color_scale,
            range_color=range_color
        )
        
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig.show()


In [9]:
# 0. LOAD BASE GEOMETRY
from shapely.geometry import Polygon

freguesias_path = "data/boundaries/lisboa_freguesias_oficial.geojson"
freguesias_gdf = gpd.read_file(freguesias_path).to_crs("EPSG:3763")
name_col = 'Des_Simpli' if 'Des_Simpli' in freguesias_gdf.columns else 'Freguesia'
freguesias_gdf = normalize_freguesia(freguesias_gdf, name_col)

# --- MANUAL CLIP WATER (Tagus River) ---
# Since we lack a precise land mask, we approximate the river boundary to fix densities.
# Coordinates are approximate trace of the coastline.
coast_points = [
    (-9.300, 38.690), # West limit
    (-9.235, 38.691), # Belém Tower area
    (-9.180, 38.695), # Alcântara
    (-9.150, 38.703), # Terreiro do Paço
    (-9.120, 38.710), # Santa Apolónia
    (-9.100, 38.735), # Beato/Marvila
    (-9.090, 38.750), # Braço de Prata
    (-9.085, 38.790), # Parque das Nações
    (-9.085, 38.850), # North limit (river side)
    (-8.900, 38.850), # East
    (-8.900, 38.500), # South East
    (-9.300, 38.500)  # South West
]

water_poly = Polygon(coast_points)
water_gdf = gpd.GeoDataFrame({'geometry': [water_poly]}, crs="EPSG:4326").to_crs("EPSG:3763")

# Clip (Difference)
freguesias_gdf = gpd.overlay(freguesias_gdf, water_gdf, how='difference')

# Recalculate Area
freguesias_gdf['Area_km2'] = freguesias_gdf.geometry.area / 10**6

# Master DataFrame for aggregations
master_stats = freguesias_gdf[['Freguesia_Norm', 'geometry', 'Area_km2', name_col]].copy()

print("Map clipped to remove water areas. Areas recalculated.")


DataSourceError: data/boundaries/lisboa_freguesias_oficial.geojson: No such file or directory

In [ ]:
# 8. ARCHITECTURE & HERITAGE
print("--- 8. Architecture ---")
# This dataset often comes as points or polygons
arch_path = "data/culture/patrimonio-arquitetonico.json" # Guessing path/name based on previous structure
if os.path.exists(arch_path):
    arch_df = pd.read_json(arch_path)
    if 'lat' in arch_df.columns:
        arch_gdf = gpd.GeoDataFrame(arch_df, geometry=gpd.points_from_xy(arch_df.lon, arch_df.lat), crs="EPSG:4326")
        
        # 8.1 Map
        fig_arch = px.scatter_mapbox(
            arch_gdf, lat="lat", lon="lon",
            hover_data=arch_gdf.columns,
            title="8.1 Architectural Heritage",
            mapbox_style="carto-positron", zoom=11,
            color_discrete_sequence=['brown']
        )
        fig_arch.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
        fig_arch.show()

        # 8.2 Density
        arch_gdf_3763 = arch_gdf.to_crs("EPSG:3763")
        joined_arch = gpd.sjoin(arch_gdf_3763, freguesias_gdf, how="inner", predicate="within")
        arch_counts = joined_arch.groupby('Freguesia_Norm').size().reset_index(name='Arch_Count')
        
        if 'Arch_Count' in master_stats.columns:
             master_stats = master_stats.drop(columns=['Arch_Count', 'Arch_Density'])
        master_stats = master_stats.merge(arch_counts, on='Freguesia_Norm', how='left').fillna(0)
        master_stats['Arch_Density'] = master_stats['Arch_Count'] / master_stats['Area_km2']

        plot_choropleth(master_stats, 'Arch_Density', "8.2 Architectural Density", cmap='Oranges')
